## Importar librerías iniciales e instancia de modelo de chat

In [1]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import SimpleSequentialChain, LLMChain,TransformChain
f = open('/home/iabd/clave_gpt.txt')
api_key = f.read().strip()
llm = ChatOpenAI(openai_api_key=api_key)

## Importamos documentos

In [2]:
from langchain.document_loaders import WikipediaLoader

In [3]:
consulta_wikipedia = input()

In [4]:
idioma_final = input()

In [7]:
loader = WikipediaLoader(query=consulta_wikipedia,lang="es",load_max_docs=10)

In [8]:
data = loader.load()

WikipediaException: An unknown error occured: "The "srsearch" parameter must be set.". Please report it on GitHub!

In [ ]:
data[0].page_content

'El Real Madrid Club de Fútbol, más conocido simplemente como Real Madrid, es una entidad polideportiva con sede en Madrid (España). Fue registrada oficialmente como club de fútbol por sus socios el 6 de marzo de 1902 con el objeto de la práctica y desarrollo de este deporte, si bien sus orígenes datan del año 1900,[10]\u200b y su denominación de (Sociedad) Madrid Foot-ball Club de octubre de 1901, siendo el quinto club fundado en la capital.[n. 3]\u200b Tuvo a Julián Palacios y los hermanos Juan Padrós y Carlos Padrós como principales valedores de su creación.[11]\u200b La entidad adquirió su designación actual el 29 de junio de 1920 mediante el otorgamiento del título honorífico de «Real» por parte de su majestad el rey Alfonso XIII de España, quien también conferiría la distinción visual correspondiente incorporando la corona real en su emblema.[12]\u200b[13]\u200b\nIdentificado por su color blanco (del que recibe el apelativo de «blancos» o «merengues»),[1]\u200b[2]\u200b es uno de

In [ ]:
texto_entrada = data[0].page_content

# TransformChain

### Definir la función de transformación personalizada

In [ ]:
def transformer_function(inputs: dict) -> dict: #Toma de entrada un diccionario y lo devuelve con la transformación oportuna
    texto = inputs['texto']
    primer_parrafo = texto.split('\n')[0]
    return {'salida':primer_parrafo}

In [ ]:
transform_chain = TransformChain(input_variables=['texto'],
                                 output_variables=['salida'],
                                 transform=transformer_function)

## Definir la cadena secuencial

In [ ]:
#Creamos bloque LLMChain para resumir
template1 = "Crea un resumen en una línea del siguiente texto:\n{texto}"
prompt = ChatPromptTemplate.from_template(template1)
summary_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="texto_resumen")

In [ ]:
#Creamos bloque LLMChain para traducir
template2 = "Traduce a"+ idioma_final + "el siguiente texto:\n{texto}"
prompt = ChatPromptTemplate.from_template(template2)
#prompt.format_prompt(idioma=idioma_final)
translate_chain = LLMChain(llm=llm,
                     prompt=prompt,
                     output_key="texto_traducido")

In [ ]:
sequential_chain = SimpleSequentialChain(chains=[transform_chain,summary_chain,translate_chain],
                                        verbose=True)

In [ ]:
result = sequential_chain(texto_entrada)

C:\Users\Iván Pinar\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
El Real Madrid Club de Fútbol, más conocido simplemente como Real Madrid, es una entidad polideportiva con sede en Madrid (España). Fue registrada oficialmente como club de fútbol por sus socios el 6 de marzo de 1902 con el objeto de la práctica y desarrollo de este deporte, si bien sus orígenes datan del año 1900,[10]​ y su denominación de (Sociedad) Madrid Foot-ball Club de octubre de 1901, siendo el quinto club fundado en la capital.[n. 3]​ Tuvo a Julián Palacios y los hermanos Juan Padrós y Carlos Padrós como principales valedores de su creación.[11]​ La entidad adquirió su designación actual el 29 de junio de 1920 mediante el otorgamiento del título honorífico de «Real» por parte de su majestad el rey Alfonso XIII de España, quien también conferiría la distinción visual correspondiente incorporando la corona real en su emblema.[12]​[13]​
El Real Madrid Club de Fútbol, fundado en 1902 en Madrid, es uno de los clubes más importantes de